# 读取所有Chracter

In [1]:
language = 'en'

In [3]:
from collections import defaultdict
import os

if language == 'cn':
    in_path = '/ldev/tensorflow/nmt2/nmt/data/wordlevel/'
    out_path = '/ldev/tensorflow/nmt2/nmt/data/hllevel/'
    os.system('mkdir %s' % out_path)
elif language == 'en':
    in_path = '/ldev/tensorflow/nmt2/nmt/data/enword_light/'
    out_path = '/ldev/tensorflow/nmt2/nmt/data/enhl_light/'
    os.system('mkdir %s' % out_path)
elif language =='en2':
    in_path = '/ldev/tensorflow/nmt2/nmt/data/enwordlevel/'
    out_path = '/ldev/tensorflow/nmt2/nmt/data/enhllevel/'
    os.system('mkdir %s' % out_path)

words_dict = {}
chars_dict = {}
for file in ['message','response']:
    characters = defaultdict(int)
    words = defaultdict(int)
    with open(in_path+'train.'+file,'r',encoding='utf-8') as fin:
        for line in fin.readlines():
            tokens = line.strip('\n').split()
            for token in tokens:
                words[token] += 1
                for char in token:
                    characters[char] += 1
        print(len(characters))
        print(len(words))
        words_dict[file] = words
        chars_dict[file] = characters
    
    

1435
119545
1151
111301


# 写一个程序，控制Character的数量
Character 包含双份的，即包含未终结的字符

Response : Response部分，Decoder部分，字和词直接混合在一起
Message： Post部分字和词需要分开

K = 4 for Coling HL-EncDec,K  = 1 For EMNLP

In [6]:
if language == 'cn':
    min_fre = 120
else:
    min_fre = 5

def count_response(total_count,file,output_id):
    words = words_dict[file]
    chars = []
    for char in chars_dict[file].keys():
        if chars_dict[file][char] >= min_fre:
            chars.append(char)
            
    vocab = set()
    sorted_words = sorted(words.items(), key = lambda x:x[1], reverse=True)
    print('response char nums: %d' % len(chars) )
    for char in chars:
        vocab.add(char)
        
    i = 0
    while(len(vocab) < total_count):
        vocab.add(sorted_words[i][0])
        i += 1

    #'增加连接符号'
    with open(out_path+'vocab.'+str(output_id)+'.'+file,'w+',encoding='utf-8') as fout:
        fout.write('\n'.join(['@@']+list(vocab)))
    print("response : %d" % len(vocab))

# K = 4 for Coling HL-EncDec,K  = 1 For EMNLP
def count_message(total_count,file,output_id,k=4):
    words = words_dict[file]
    chars = []
    for char in chars_dict[file].keys():
        if chars_dict[file][char] >= min_fre:
            chars.append(char)
    vocab = set()
    sorted_words = sorted(words.items(), key = lambda x:x[1], reverse=True)
    print('message char nums: %d' % len(chars) )
    for i in range(total_count-len(chars)):
        vocab.add(sorted_words[i][0])   
    if '@@' in vocab or ' ' in vocab:
        print('Error')
    with open(out_path+'vocab.'+str(output_id)+'.'+file,'w+',encoding='utf-8') as fout:
        fout.write('\n'.join(list(vocab)))
    print("message : %d" % len(vocab))


count_message(15000,'message',15000,2)
count_response(15000,'response',15000)


message char nums: 617
message : 14383
response char nums: 508
response : 15000


# 读取对应的文件，然后生成相应的混合表示

In [7]:
for file in ['response']:
    max_len = 0
    for vocab_size in [15000]:
        vocab = set()
        with open(out_path+'vocab.'+str(vocab_size)+'.'+file,'r+',encoding='utf-8') as fin:
            for line in fin.readlines():
                vocab.add(line.strip())
        for prefix  in ['train','test','dev']:
            sen_len_counter = defaultdict(int)
            with open(out_path+prefix+'.'+str(vocab_size)+'.'+file,'w',encoding='utf-8') as fout:
                with open(in_path+prefix+'.'+file,'r',encoding='utf-8') as fin:
                    total_len = 0
                    for line in fin.readlines():
                        res = []
                        for token in line.strip('\n').split():
                            
                            if token not in vocab:
                                for char in token[:-1]:
                                    res.append(char)
                                    res.append('@@')
                                res.append(token[-1])
                                    
                            else:
                                res.append(token)
                        max_len = max(len(res), max_len)
                        res = res[0:70]
                        fout.write(' '.join(res)+'\n')
                        sen_len_counter[len(res)] += 1
                        total_len += 1
                    print(total_len)
                print(max_len)
            if prefix == 'train':
                print('items_per_sentence: %s %s' % (file,vocab_size))
                sorted_length = sorted(sen_len_counter.items(), key = lambda x:x[0])
                total_counter = 0
                line_str = ''
                for lens,counter in sorted_length:
                    total_counter += counter
                    line_str += '%d %.2f ' % (lens, total_counter/total_len)
                print (line_str)

for file in ['message']:
    for vocab_size in [15000]:
        vocab = set()
        with open(out_path+'vocab.'+str(vocab_size)+'.'+file,'r+',encoding='utf-8') as fin:
            for line in fin.readlines():
                vocab.add(line.strip())
        for prefix  in ['train','test','dev']:
            sen_len_counter = defaultdict(int)
            max_len = 0
            with open(out_path+prefix+'.'+str(vocab_size)+'.'+file,'w',encoding='utf-8') as fout:
                with open(in_path+prefix+'.'+file,'r',encoding='utf-8') as fin:
                    total_len = 0
                    for line in fin.readlines():
                        res = []
                        for token in line.strip('\n').split():
                                res.append(token)
                        max_len = max(len(res), max_len)
                        res = res[0:70]
                        fout.write(' '.join(res)+'\n')
                        sen_len_counter[len(res)] += 1
                        total_len += 1
                    print(total_len)
            print(max_len)
            if prefix == 'train':
                print('items_per_sentence: %s %s' % (file,vocab_size))
                sorted_length = sorted(sen_len_counter.items(), key = lambda x:x[0])
                total_counter = 0
                line_str = ''
                for lens,counter in sorted_length:
                    total_counter += counter
                    line_str += '%d %.2f ' % (lens, total_counter/total_len)
                print (line_str)

195561
226
items_per_sentence: response 15000
4 0.06 5 0.12 6 0.18 7 0.23 8 0.28 9 0.33 10 0.37 11 0.40 12 0.44 13 0.47 14 0.49 15 0.52 16 0.55 17 0.57 18 0.59 19 0.62 20 0.64 21 0.65 22 0.67 23 0.68 24 0.70 25 0.71 26 0.73 27 0.74 28 0.75 29 0.77 30 0.78 31 0.79 32 0.81 33 0.82 34 0.83 35 0.84 36 0.85 37 0.86 38 0.87 39 0.87 40 0.88 41 0.89 42 0.89 43 0.90 44 0.90 45 0.91 46 0.92 47 0.92 48 0.92 49 0.93 50 0.93 51 0.94 52 0.94 53 0.95 54 0.95 55 0.95 56 0.95 57 0.96 58 0.96 59 0.96 60 0.97 61 0.97 62 0.97 63 0.97 64 0.97 65 0.97 66 0.98 67 0.98 68 0.98 69 0.98 70 1.00 
10000
226
10000
226
195561
20
items_per_sentence: message 15000
4 0.04 5 0.09 6 0.15 7 0.22 8 0.28 9 0.34 10 0.41 11 0.47 12 0.54 13 0.60 14 0.66 15 0.72 16 0.78 17 0.84 18 0.89 19 0.95 20 1.00 
10000
20
10000
20


# Segment

In [10]:
# Configs including a end
if language == 'cn':
    seg_len = 5
else:
    seg_len = 8 # 10已经足够了 99%
seg_end = '<#>'
seg_pad = '<_>'
seg_separator = '\t'
seg_inter_separator = ' '
vocab_nums = [15000]
from collections import defaultdict
# read the vocabs and add all characters in the vocab
def add_chars_to_vocab(file_path,vocab_path, word_num = -1):
    with open(file_path,'r+',encoding='utf-8') as fin:
        with open(vocab_path+'_tmp','w+',encoding='utf-8') as fout:
            lines = fin.readlines()
            print('original num: %d' % len(lines))
            vocab = defaultdict(int)
            vocab[seg_end] = 99999
            vocab[seg_pad] = 99999
            
            # first 3 lines are special tokens
            if word_num != -1:
                lines = lines[0:0+word_num]
            else:
                lines = lines[0:]
            for line in lines:
                line = line.strip('\n')
                line=line.replace(' ','')
                line=line.replace('\t','')
                #vocab.add(word)
                if len(line) > 1:
                    for char in line:
                        vocab[char] += 1
                
            for token in vocab.keys():
                if vocab[token] > 0:
                    fout.write(token+'\n')
                #print(token)
            print('new num: %d' % len(vocab))
            
    with open(vocab_path+'_tmp','r+',encoding='utf-8') as fin:
        with open(vocab_path+'_seg','w+',encoding='utf-8') as fout:
            lines = fin.readlines()
            print('original num: %d' % len(lines))
            vocab = set()
            vocab.add(seg_end)
            vocab.add(seg_pad)
            
            # first 3 lines are special tokens
            if word_num != -1:
                lines = lines[0:0+word_num]
            else:
                lines = lines[0:]
            for line in lines:
                line = line.strip('\n')
                #vocab.add(word)
                if len(line) > 0:
                    for char in line:
                        vocab.add(str(char))
            count = 0
            for token in vocab:
                if len(token) > 0:
                    fout.write(token+'\n')
                    count += 1
                #print(token)
            print('new num: %d' % count)



def convert_to_seg_file(file_path, seg_len,vocab):
    with open(file_path, 'r+', encoding='utf-8') as fin:
        nfile_path = file_path
        with open(nfile_path + '_seg', 'w+', encoding='utf-8') as fout,open(nfile_path + '_seg_len', 'w+', encoding='utf-8') as flout:
            lines = fin.readlines()
            for line in lines:
                items = line.strip('\n').split(' ')
                seg_items = []
                for item in items:
                    item = list(item)
                    item = item[0:seg_len - 1]
                    item.append(seg_end)
                    while len(item) != seg_len:
                        item.append(seg_pad)
                    seg_items.append(seg_inter_separator.join(item))
                flout.write(' '.join([str(min(seg_len,len(x)+1)) for x in items]) + '\n')
                fout.write(seg_separator.join(seg_items) + '\n')

for vocab in vocab_nums:
    # TODO 所有
    add_chars_to_vocab(out_path+r'train.%d.message' % (vocab), out_path+r'vocab.%d.message' % vocab)
    add_chars_to_vocab(out_path+r'train.%d.response'% (vocab), out_path+r'vocab.%d.response'% vocab)
    convert_to_seg_file(out_path+r'train.%d.message'% (vocab), seg_len,vocab)
    convert_to_seg_file(out_path+r'train.%d.response'% (vocab), seg_len,vocab)
    convert_to_seg_file(out_path+r'test.%d.message'% (vocab), seg_len,vocab)
    convert_to_seg_file(out_path+r'test.%d.response'% (vocab), seg_len,vocab)
    convert_to_seg_file(out_path+r'dev.%d.message'% (vocab), seg_len,vocab)
    convert_to_seg_file(out_path+r'dev.%d.response'% (vocab), seg_len,vocab)




original num: 195561
new num: 1437
original num: 1437
new num: 1439
original num: 195561
new num: 1143
original num: 1143
new num: 1145
